In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：使用TF Serving容器进行在线预测的自定义训练图像分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_image_classification_online_tfserving.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_image_classification_online_tfserving.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库来训练和部署自定义图像分类模型以进行在线预测，使用TF Serving容器进行预测。

数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。您将使用的数据集版本已经内置在TensorFlow中。训练模型预测图像属于十个类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车。

### 目标

在本教程中，您将使用Vertex客户端库在Google预构建的Docker容器中的Python脚本中创建自定义模型，然后使用TF Serving容器对部署的模型进行预测。您也可以选择使用`gcloud`命令行工具或在Google Cloud控制台上在线创建自定义模型。

执行的步骤包括：

- 为训练模型创建一个Vertex自定义作业。
- 创建一个 `TrainingPipeline` 资源。
- 训练一个TensorFlow模型。
- 检索和加载模型工件。
- 查看模型评估。
- 将模型上传为一个Vertex `Model` 资源。
- 将`Model` 资源部署到一个带有TF Serving容器的serving `Endpoint` 资源。
- 进行预测。
- 取消部署`Model` 资源。

成本

本教程使用Google Cloud（GCP）的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/) 来根据您的预计使用量生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装*google-cloud-storage*库的最新GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动核心

在安装了Vertex客户端库和Google *cloud-storage*后，您需要重新启动笔记本核心，以便它能找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果有这个选项，请确保在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **Runtime > Change Runtime Type > GPU**

### 设置您的 Google Cloud 项目

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个帐户时，您会获得 $300 的免费信贷，用于支付计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex APIs 和 Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 Google Cloud Notebook 中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**: Jupyter 运行以 `!` 为前缀的行作为 shell 命令，并将以 `$` 为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于笔记本的其余部分操作。以下是Vertex支持的区域。我们建议您选择最靠近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您不可以使用多区域存储桶进行Vertex的训练。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行直播教程会话，您可能会使用共享的测试帐户或项目。为了避免在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并附加到将在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###验证您的Google Cloud帐户

**如果您正在使用Google Cloud笔记本**，您的环境已经经过验证。请跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并在提示时按照说明进行oAuth验证您的账户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**字段中输入一个名称，然后点击**创建**。

在**授予该服务账户对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入"Vertex"，选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，选择**Storage Object Admin**。

点击创建。包含您的密钥的JSON文件将下载到本地环境中。

在下面的单元格中将您的服务账户密钥的路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论你使用的笔记本环境如何，都需要以下步骤。**

当您使用Vertex客户端库提交一个自定义训练作业时，您需要将包含训练代码的Python包上传到一个云存储桶中。Vertex会从这个包中运行代码。在本教程中，Vertex还会在同一个存储桶中保存从您的作业结果中产生的训练模型。然后，您可以根据这个输出创建一个基于此的`Endpoint`资源，以便提供在线预测。

请在下面设置您的云存储桶的名称。存储桶的名称必须在所有的Google Cloud项目中是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证对其的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库并定义常量

#### 导入Vertex客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、流水线和端点服务的顶点 API 服务端点。
- `PARENT`：顶点位置根路径，用于数据集、模型、作业、流水线和端点资源。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

### 安装Docker（Colab或本地）

如果您在使用Google云笔记本，Docker已经安装好了。请跳过以下步骤。

默认情况下，Colab上没有安装Docker。如果您正在运行colab，那么您需要按照以下步骤安装Docker。

In [ ]:
if "google.colab" in sys.modules:
    ! sudo apt update
    ! sudo apt install apt-transport-https ca-certificates curl software-properties-common
    ! curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -
    ! sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable"
    ! sudo apt update
    ! sudo apt install docker-ce

启动Docker服务

启动`docker`服务。

In [ ]:
if "google.colab" in sys.modules:
    ! sudo service docker start

硬件加速器

设置硬件加速器（例如，GPU）用于训练和预测。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`以使用支持GPU的容器映像和分配给虚拟机（VM）实例的GPU数量。例如，要使用一个具有4个 Nvidia Telsa K80 GPU分配给每个 VM 的 GPU 容器映像，您将指定：

(aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

对于 GPU，可用的加速器包括:
- aip.AcceleratorType.NVIDIA_TESLA_K80
- aip.AcceleratorType.NVIDIA_TESLA_P100
- aip.AcceleratorType.NVIDIA_TESLA_P4
- aip.AcceleratorType.NVIDIA_TESLA_T4
- aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`以使用一个运行在 CPU 上的容器映像。

*注*: 早于 2.3 版本的 TensorFlow 不支持 GPU，将无法在本教程中加载自定义模型。这是一个已知问题，在 TF 2.3 中已经修复 -- 这是由于在服务函数中生成的静态图操作引起的问题。如果您在自己的自定义模型中遇到此问题，请使用支持 GPU 的 TF 2.3 的容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 容器（Docker）镜像

接下来，我们将为训练设置Docker容器镜像。

- 将变量`TF`设置为容器镜像的TensorFlow版本。例如，`2-1`表示版本2.1，`1-15`表示版本1.15。以下列表显示了一些预构建的可用镜像：

 - TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
 - TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
 - TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
 - TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
 - TensorFlow 2.4
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
 - XGBoost
   - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
 - Scikit-learn
   - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
 - Pytorch
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

查看最新列表，请参见[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)

#### 用于预测的容器（Docker）镜像

接下来，您设置 TF Serving 的 Docker 容器镜像用于预测。

1. 从 Docker Hub 拉取适用于 TF Serving 的相应 CPU 或 GPU Docker 镜像。
2. 为将镜像注册到云容器注册表（gcr.io）创建一个标签。
3. 将镜像注册到云容器注册表。

有关使用 TF Serving 容器的更多详细信息，请参见 [TF Serving](https://www.tensorflow.org/tfx/serving/docker)。

In [ ]:
if DEPLOY_GPU:
    ! docker pull tensorflow/serving:latest-gpu
    DEPLOY_IMAGE = "gcr.io/" + PROJECT_ID + "/tf_serving:gpu"
else:
    ! docker pull tensorflow/serving:latest
    DEPLOY_IMAGE = "gcr.io/" + PROJECT_ID + "/tf_serving"

! docker tag tensorflow/serving $DEPLOY_IMAGE
! docker push $DEPLOY_IMAGE

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `TRAIN_COMPUTE` 和 `DEPLOY_COMPUTE` 来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个 vCPU 3.75GB 的内存。
     - `n1-highmem`: 每个 vCPU 6.5GB 的内存。
     - `n1-highcpu`: 每个 vCPU 0.9GB 的内存。
 - `vCPUs`: vCPUs 的数量为 \[2, 4, 8, 16, 32, 64, 96 \]

*注意：以下机器类型不支持用于训练：*

 - `standard`: 2 vCPUs
 - `highcpu`: 2、4 和 8 vCPUs

*注意：您也可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持 GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在你已经准备好开始创建自己的定制模型并训练CIFAR10了。

## 设置客户端

Vertex客户端库作为客户端/服务器模型工作。在您的一侧（Python脚本），您将创建一个客户端，该客户端将向Vertex服务器发送请求并接收响应。

在本教程中，您将为工作流程的不同步骤使用不同的客户端。因此，请提前设置它们。

- 用于`Model`资源的模型服务。
- 用于部署的终端服务。
- 用于批处理作业和自定义训练的作业服务。
- 用于提供服务的预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

训练模型

您可以使用容器图像的两种方式来训练自定义模型：

- **使用Google Cloud预构建容器**。如果您使用预构建容器，您还需要指定要安装到容器图像中的Python软件包。此Python软件包包含用于训练自定义模型的代码。

- **使用您自己的自定义容器图像**。如果您使用自己的容器，容器需要包含用于训练自定义模型的代码。

## 准备您的定制工作规范

现在您的客户准备好了，您的第一步是为定制培训工作创建一个工作规范。工作规范将包括以下内容：

- `worker_pool_spec`：用于培训的机器类型规范及数量（单个或分布式）
- `python_package_spec`：与预构建容器一起安装的Python包规范。

### 准备您的机器规格

现在定义您定制训练作业的机器规格。这告诉 Vertex 将为训练提供哪种类型的机器实例。
- `machine_type`：要提供的 GCP 实例的类型 -- 例如，n1-standard-8。
- `accelerator_type`：硬件加速器的类型，如果有的话。在本教程中，如果您之前设置了变量`TRAIN_GPU != None`，则表示您正在使用 GPU；否则您将使用 CPU。
- `accelerator_count`：加速器数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的硬盘规格

（可选）现在为您的自定义训练工作定义硬盘规格。这告诉 Vertex 在每个机器实例中为训练提供什么类型和大小的硬盘。

  - `boot_disk_type`：固态硬盘（SSD）或标准硬盘。固态硬盘（SSD）速度更快，标准硬盘更便宜。默认为固态硬盘（SSD）。
  - `boot_disk_size_gb`：硬盘的大小（以GB为单位）。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

### 定义工作进程池规范

接下来，您将定义自定义训练任务的工作进程池规范。工作进程池规范将包括以下内容：

- `replica_count`：要提供的此机器类型的实例数。
- `machine_spec`：硬件规格。
- `disk_spec`：（可选）磁盘存储规格。

- `python_package`：要安装在VM实例上的Python训练包以及要调用的Python模块，以及Python模块的命令行参数。

现在让我们深入了解Python包规范：

- `executor_image_spec`：这是为您的自定义训练任务配置的docker镜像。

- `package_uris`：这是要安装在提供实例上的Python训练包的位置（URI）列表。这些位置需要在Cloud Storage存储桶中。这些可以是单独的Python文件，也可以是整个软件包的zip（存档）。在后一种情况下，作业服务将解压（解档）内容到docker镜像中。

- `python_module`：要调用以运行自定义训练任务的Python模块（脚本）。在此示例中，您将调用`trainer.task.py` -- 请注意不需要添加`.py`后缀。

- `args`：要传递给相应Python模块的命令行参数。在此示例中，您将设置：
  - `"--model-dir=" + MODEL_DIR`：存储模型工件的Cloud Storage位置。告诉训练脚本保存模型工件的两种方法：
     - 直接：将Cloud Storage位置作为命令行参数传递给您的训练脚本（设置变量`DIRECT = True`），
     - 间接：服务将Cloud Storage位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本（设置变量`DIRECT = False`）。在这种情况下，在作业规范中告诉服务模型工件位置。
  - `"--epochs=" + EPOCHS`：训练的周期数。
  - `"--steps=" + STEPS`：每周期的步骤（批次）数。
  - `"--distribute=" + TRAIN_STRATEGY"`：用于单一或分布式训练的训练分发策略。
     - `"single"`：单一设备。
     - `"mirror"`：单个计算实例上的所有GPU设备。
     - `"multi"`：所有计算实例上的所有GPU设备。

*注意*：对于TF Serving，`MODEL_DIR`必须以数字为子文件夹结尾，例如1。

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = "{}/{}/1".format(BUCKET_NAME, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_cifar10.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 组装一个工作规格

现在组装自定义工作规格的完整描述：

- `display_name`：您为这个自定义工作指定的可读名称。
- `job_spec`：自定义工作规格的规范。
    - `worker_pool_specs`：机器VM实例的规范。
    - `base_output_directory`：这告诉服务将模型制件保存在哪个云存储位置（当变量 `DIRECT = False` 时）。然后服务将该位置传递给训练脚本作为环境变量 `AIP_MODEL_DIR`，路径的格式将为：

                <output_uri_prefix>/model

In [ ]:
if DIRECT:
    job_spec = {"worker_pool_specs": worker_pool_spec}
else:
    job_spec = {
        "worker_pool_specs": worker_pool_spec,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    }

custom_job = {"display_name": JOB_NAME, "job_spec": job_spec}

### 检查培训套件

#### 套件布局

在开始培训之前，您将查看如何为自定义培训工作流程组装 Python 包。解压后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是用于将包安装到 Docker 镜像的操作环境的指令。

文件 `trainer/task.py` 是用于执行自定义培训工作的 Python 脚本。*注意*，当我们在工作池规范中引用它时，我们将目录斜杠替换为点 (`trainer.task`) 并省略文件后缀(`.py`)。

#### 套件组装

在下面的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 文件内容

在下一个单元格中，您将编写训练脚本 task.py 的内容。我们不会详细展开，只是让您浏览一下。总结如下：

- 从命令行获取要保存模型工件的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从 TF Datasets（tfds）中加载 CIFAR10 数据集。
- 使用 TF.Keras 模型 API 构建模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分发策略。
- 使用参数 `args.epochs` 和 `args.steps` 训练模型（`fit()`）。
- 将训练好的模型保存到指定的模型目录中（`save(args.model_dir)`）。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets_unbatched():

  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model


# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

将培训脚本存储在您的云存储桶上

接下来，您将培训文件夹打包成压缩的tar球，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_cifar10.tar.gz

### 训练模型

现在开始在Vertex上训练您的自定义训练任务。使用这个辅助函数`create_custom_job`，它接受以下参数：

- `custom_job`：自定义作业的规范。

辅助函数调用作业客户端服务的`create_custom_job`方法，具有以下参数：

- `parent`：指向`Dataset`、`Model`和`Endpoint`资源的Vertex位置路径。
- `custom_job`：自定义作业的规范。

您将显示返回对象中的一些字段，其中最感兴趣的两个字段是：

`response.name`：分配给这个自定义训练任务的Vertex完全限定标识符。您需要保存这个标识符以便在后续步骤中使用。

`response.state`：自定义训练作业的当前状态。

In [ ]:
def create_custom_job(custom_job):
    response = clients["job"].create_custom_job(parent=PARENT, custom_job=custom_job)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_custom_job(custom_job)

现在获取您创建的自定义工作的唯一标识符。

In [ ]:
# The full unique ID for the custom job
job_id = response.name
# The short numeric ID for the custom job
job_short_id = job_id.split("/")[-1]

print(job_id)

获取有关自定义作业的信息

接下来，使用这个辅助函数`get_custom_job`，它接受以下参数：

- `name`：自定义作业的Vertex完全限定标识符。

辅助函数调用作业客户端服务的`get_custom_job`方法，参数如下：

- `name`：自定义作业的Vertex完全限定标识符。

如果你记得，当你调用`create_custom_job`方法时，在`response.name`字段中获取了自定义作业的Vertex完全限定标识符，并将标识符保存在变量`job_id`中。

In [ ]:
def get_custom_job(name, silent=False):
    response = clients["job"].get_custom_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_custom_job(job_id)

# 部署

训练上述模型可能需要长达20分钟的时间。

一旦您的模型训练完成，您可以通过将`end_time`减去`start_time`来计算训练模型所花费的实际时间。对于您的模型，我们需要知道保存模型的位置，Python脚本将保存在您本地的云存储桶中的`MODEL_DIR + '/saved_model.pb'`。

In [ ]:
while True:
    response = get_custom_job(job_id, True)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_path_to_deploy = None
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        model_path_to_deploy = MODEL_DIR
        print("Training Time:", response.update_time - response.create_time)
        break
    time.sleep(60)

print("model_to_deploy:", model_path_to_deploy)

加载已保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud 存储桶中。现在从 Cloud 存储桶中加载它，然后您可以做一些事情，比如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras 的 `model.load_model()` 方法，将保存模型的 Cloud 存储路径传递给它 -- 由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在找出模型有多好。

### 加载评估数据

您将从`tf.keras.datasets`中加载CIFAR10测试（留出）数据，使用`load_data（）`方法。这将返回数据集作为两个元素的元组。第一个元素是训练数据，第二个是测试数据。每个元素也是两个元素的元组：图像数据和相应的标签。

您不需要训练数据，因此我们将其加载为`(_, _)`。

在将数据放入评估之前，您需要对其进行预处理：

x_test:
1. 将像素数据归一化（重新缩放），方式是将每个像素除以255。这将用0到1之间的32位浮点数替换每个单字节整数像素。

y_test:<br/>
2. 标签目前是标量（稀疏）。如果您回顾`trainer/task.py`脚本中的`compile（）`步骤，您会发现它是为稀疏标签编译的。所以我们不需要做任何额外的工作。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10

(_, _), (x_test, y_test) = cifar10.load_data()
x_test = (x_test / 255.0).astype(np.float32)

print(x_test.shape, y_test.shape)

### 进行模型评估

现在评估定制工作中的模型表现如何。

In [ ]:
model.evaluate(x_test, y_test)

## 上传模型以提供服务

在接下来的步骤中，您将从自定义作业上传您的TF.Keras模型到Vertex `Model`服务，这将为您的自定义模型创建一个Vertex `Model`资源。在上传过程中，您需要定义一个提供服务的函数，以将数据转换为您的模型期望的格式。如果您将编码的数据发送给Vertex，您的提供服务函数将确保在将数据作为输入传递给您的模型之前，在模型服务器上对数据进行解码。

### 提供服务函数如何工作

当您向在线预测服务器发送请求时，该请求将由一个HTTP服务器接收。HTTP服务器从HTTP请求内容中提取预测请求。提取的预测请求将被转发给提供服务函数。对于Google预构建的预测容器，请求内容将作为`tf.string`传递给提供服务函数。

提供服务函数包括两部分：

- `预处理函数`：
  - 将输入（`tf.string`）转换为底层模型的输入形状和数据类型（动态图）。
  - 执行与训练底层模型期间相同的数据预处理 -- 例如标准化、缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序所期望的格式 -- 例如压缩输出。
  - 对接收应用程序的输出进行打包 -- 例如添加标题，生成JSON对象等。

预处理和后处理函数均被转换为与模型融合的静态图。来自底层模型的输出被传递给后处理函数。后处理函数将转换/打包后的输出传递回HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

构建TF.Keras模型的提供服务函数时需要考虑的一点是它们作为静态图运行。这意味着您不能使用需要动态图的TF图操作。如果这样做，您将在编译提供服务函数时收到错误提示，指出您正在使用不受支持的EagerTensor。

### 图像数据的服务功能

为了将图像传递给预测服务，您需要将压缩的（例如JPEG）图像字节编码为 base 64 —— 这样做可以确保在通过网络传输二进制数据时内容不会被修改。由于这个部署的模型期望输入数据为原始（未压缩的）字节，您需要确保 base 64 编码的数据在传递给部署的模型之前被转换回原始字节。

为了解决这个问题，定义一个服务功能（`serving_fn`）并将其附加到模型作为预处理步骤。添加一个`@tf.function`装饰器，这样服务功能将与底层模型融合在一起（而不是在 CPU 上游进行处理）。

当您发送预测或解释请求时，请求的内容将被 base 64 解码为一个 Tensorflow 字符串（`tf.string`），然后传递给服务功能（`serving_fn`）。服务功能将预处理`tf.string`为原始（未压缩的）numpy字节（`preprocess_fn`），以满足模型的输入要求：
- `io.decode_jpeg`- 解压缩 JPG 图像，返回一个具有三个通道（RGB）的 Tensorflow 张量。
- `image.convert_image_dtype` - 将整数像素值转换为 32 位浮点数，并在 0 和 1 之间重新缩放像素数据。
- `image.resize` - 调整图像大小以匹配模型的输入形状。

此时，数据可以传递到模型（`m_call`）。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(32, 32))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    model, model_path_to_deploy, signatures={"serving_default": serving_fn}
)

获取服务函数签名

您可以通过重新加载模型到内存中，并查询每个层对应的签名来获取模型的输入和输出层的签名。

对于您的目的，您需要服务函数的签名。为什么呢？当我们将数据作为HTTP请求包发送进行预测时，图像数据是base64编码的，而我们的TF.Keras模型需要numpy输入。您的服务函数将执行从base64到numpy数组的转换。

在进行预测请求时，您需要将请求路由到服务函数而不是模型，因此您需要知道服务函数的输入层名称--在您进行预测请求时将会用到。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

### 上传模型

使用这个帮助函数`upload_model`来上传您的模型，存储在SavedModel格式中，到`Model`服务，该服务将为您的模型实例化一个Vertex `Model`资源实例。一旦您完成了这个步骤，您就可以像任何其他Vertex `Model`资源实例一样使用`Model`资源实例，例如部署到用于提供预测的`Endpoint`资源。

这个帮助函数接受以下参数：

- `display_name`：`Endpoint`服务的可读名称。
- `image_uri`：模型部署的容器镜像。
- `model_uri`：我们的SavedModel艺术品在Cloud Storage中的路径。对于本教程，这是`trainer/task.py`保存模型艺术品的Cloud Storage位置，我们在变量`MODEL_DIR`中指定。

这个帮助函数调用`Model`客户端服务的方法`upload_model`，该方法接受以下参数：

- `parent`：`Dataset`，`Model`和`Endpoint`资源的Vertex位置根路径。
- `model`：Vertex`Model`资源实例的规范。

现在让我们深入了解Vertex模型规范`model`。这是一个包含以下字段的字典对象：

- `display_name`：`Model`资源的可读名称。
- `metadata_schema_uri`：由于您的模型是构建的而没有Vertex`Dataset`资源，您将此字段留空(`''`)。
- `artificat_uri`：模型以SavedModel格式存储在Cloud Storage中的路径。
- `container_spec`：这是将在端点上安装的Docker容器的规范，从中模型将提供预测。在本教程中，您使用一个TF Serving作为自定义预测容器。由于这是一个自定义容器，您需要告诉Vertex如何启动并与服务二进制通信：
    - `command`：要启动的服务二进制（HTTP服务器）。
    - `args`：要传递给服务二进制的参数。对于TF Serving，所需的参数是：
        - `--model_name`：分配给模型的可读名称。
        - `--model_base_name`：容器中存储模型艺术品的位置。Vertex服务将变量$(AIP_STORAGE_URI)设置为服务在容器中安装模型艺术品的位置。
        - `--rest_api_port`：用于发送基于REST的预测请求的端口。可以是8080或8501（TF Serving的默认端口）。
        - `--port`：用于发送基于gRPC的预测请求的端口。对于TF Serving，应该是8500。

    - `health_route`：服务定期ping以获取响应以验证服务二进制正在运行的URL。对于TF Serving，这将是/v1/models/[model_name]。
    - `predict_route`：服务将基于REST的预测请求路由到的URL。对于TF Serving，这将是/v1/models/[model_name]:predict。

将模型上传到Vertex Model资源会返回一个长时间运行的操作，因为可能需要一些时间。您调用`response.result()`，这是一个同步调用，当Vertex Model资源就绪时会返回。

帮助函数将返回对应的Vertex Model实例`upload_model_response.model`的完全限定标识符。您将保存这个标识符以供后续步骤使用，在变量`model_to_deploy_id`中。

*注意*：您需要删除模型路径的末尾数字子文件夹（例如，/1）以上传。Vertex服务将上传包含模型艺术品的子文件夹上方的父文件夹 -- 这是TensorFlow服务二进制预期的。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE
MODEL_NAME = "cifar10-" + TIMESTAMP


def upload_model(display_name, image_uri, model_uri):
    model = {
        "display_name": display_name,
        "metadata_schema_uri": "",
        "artifact_uri": model_uri,
        "container_spec": {
            "image_uri": image_uri,
            "command": ["/usr/bin/tensorflow_model_server"],
            "args": [
                "--model_name=" + MODEL_NAME,
                "--model_base_path=" + "$(AIP_STORAGE_URI)",
                "--rest_api_port=8080",
                "--port=8500",
                "--file_system_poll_wait_seconds=31540000",
            ],
            "health_route": "/v1/models/" + MODEL_NAME,
            "predict_route": "/v1/models/" + MODEL_NAME + ":predict",
        },
    }
    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model(
    "cifar10-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy[:-2]
)

获取`Model`资源信息

现在让我们获取只有您的模型的模型信息。使用这个辅助函数`get_model`，带有以下参数：

- `name`：用于`Model`资源的Vertex唯一标识符。

这个辅助函数调用Vertex `Model`客户端服务的方法`get_model`，带有以下参数：

- `name`：用于`Model`资源的Vertex唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

部署`Model`资源

现在部署经过训练的Vertex自定义`Model`资源。 这需要两个步骤：

1. 为部署`Model`资源创建一个`Endpoint` 资源。

2. 将`Model`资源部署到`Endpoint`资源。

### 创建一个`Endpoint`资源

使用这个辅助函数`create_endpoint`来创建一个用于部署模型以提供预测的终端，需要以下参数：

- `display_name`: `Endpoint`资源的人类可读名称。

这个辅助函数使用终端客户端服务的`create_endpoint`方法，该方法需要以下参数：

- `display_name`: `Endpoint`资源的人类可读名称。

创建`Endpoint`资源将返回一个长时间运行的操作，因为为了提供服务，可能需要一些时间来准备`Endpoint`资源。您可以调用`response.result()`，这是一个同步调用，当Endpoint资源准备就绪时返回。辅助函数返回`Endpoint`资源的Vertex完全合格标识符: `response.name`。

In [ ]:
ENDPOINT_NAME = "cifar10_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的“端点”资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例的扩展

在处理您的在线预测请求时，您有几种计算实例扩展的选择：

- 单一实例：在线预测请求在单个计算实例上处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩展：在线预测请求在您手动指定的固定数量的计算实例之间分配。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同数量的节点。当模型首次部署到实例时，固定数量的计算实例将被提供，并且在线预测请求将均匀分布在它们之间。

- 自动扩展：在线预测请求在可扩展数量的计算实例之间分配。
  - 根据负载情况，将计算实例的最小（`MIN_NODES`）数量设置为模型首次部署和撤销时要提供的实例数，并将最大（`MAX_NODES`）数量设置为要提供的计算实例数。

计算实例的最小数量对应于字段`min_replica_count`，而最大数量对应于字段`max_replica_count`，在您随后的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署 `Model` 资源到 `Endpoint` 资源

使用这个辅助函数 `deploy_model`，将从训练流水线中上传（部署）的 `Model` 资源部署到为提供预测而创建的 `Endpoint` 资源，使用以下参数：

- `model`: 要上传（部署）的 Vertex 完全限定模型标识符。
- `deploy_model_display_name`: 部署模型的人类可读名称。
- `endpoint`: 要将模型部署到的 Vertex 完全限定端点标识符。

辅助函数调用 `Endpoint` 客户端服务的方法 `deploy_model`，该方法接受以下参数：

- `endpoint`: 要将 `Model` 资源部署到的 Vertex 完全限定 `Endpoint` 资源标识符。
- `deployed_model`: 部署模型的要求规范。
- `traffic_split`: 进入终端点的流量百分比，指定为一个或多个键/值对的字典。
   - 如果只有一个模型，则指定为 **{ "0": 100 }**，其中 "0" 指的是上传的这个模型，100 表示流量的 100%。
   - 如果终端点上有现有模型，流量将被分配到这些模型，使用 `model_id` 指定为 **{ "0": percent, model_id: percent, ... }**，其中 `model_id` 是要部署到终端点的现有模型的模型标识符。百分比必须加起来等于 100。

现在让我们深入了解 `deployed_model` 参数。此参数被指定为一个具有最低必要字段的 Python 字典：

- `model`: 要部署的（上传的） Vertex 完全限定模型标识符。
- `display_name`: 部署模型的人类可读名称。
- `disable_container_logging`: 这将禁用容器事件的日志记录，例如执行失败（默认情况下启用容器日志记录）。容器日志记录通常在调试部署时启用，然后在投入生产时禁用。
- `dedicated_resources`: 这指的是为了提供预测请求而扩展的计算实例（副本）的数量。
  - `machine_spec`: 要预留的计算实例。使用您之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只分配一个 CPU。
  - `min_replica_count`: 最初预留的计算实例数量，您之前将其设置为变量 `MIN_NODES`。
  - `max_replica_count`: 可扩展到的最大计算实例数量，您之前将其设置为变量 `MAX_NODES`。

#### 流量分割

现在让我们深入了解 `traffic_split` 参数。此参数被指定为一个Python字典。一开始可能有点混乱。让我解释一下，您可以将一个模型的多个实例部署到一个端点，然后设置每个实例的流量百分比。

为什么要这样做？也许您已经在生产中部署了以前的版本 -- 让我们称之为 v1。在 v2 上获得了更好的模型评估，但在部署到生产环境之前您并不确定它是否真的更好。因此，在流量分割的情况下，您可能希望将 v2 部署到与 v1 相同的端点，但只分配给它，比如说 10% 的流量。这样，您可以在不影响大多数用户的情况下监视其性能直到最终决定。

#### 响应

该方法将返回一个长时间运行的操作 `response`。我们将通过调用 `response.result()` 来等待操作同步完成，并一直阻塞直到模型部署完毕。如果这是第一次将模型部署到端点，则可能需要额外几分钟来完成资源的预留。

In [ ]:
DEPLOYED_NAME = "cifar10_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "disable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

发出一个在线预测请求

现在对您部署的模型进行在线预测。

获取测试项目

您将使用数据集中测试（留置）部分的示例作为测试项目。

In [ ]:
test_image = x_test[0]
test_label = y_test[0]
print(test_image.shape)

### 准备请求内容
你将会发送CIFAR10图片作为压缩的JPG图片，而不是原始未压缩的字节：

- `cv2.imwrite`: 使用openCV将未压缩的图片写入磁盘，保存为压缩的JPEG图片。
 - 将图像数据从\[0,1)范围反向归一化到[0,255)。
 - 将32位浮点数值转换为8位无符号整数。
- `tf.io.read_file`: 将压缩的JPG图片作为原始字节读取到内存中。
- `base64.b64encode`: 将原始字节编码为Base64编码的字符串。

In [ ]:
import base64

import cv2

cv2.imwrite("tmp.jpg", (test_image * 255).astype(np.uint8))

bytes = tf.io.read_file("tmp.jpg")
b64str = base64.b64encode(bytes.numpy()).decode("utf-8")

发送预测请求

好的，现在你有一张测试图片。使用这个辅助函数`predict_image`，它接受以下参数：

- `image`：作为numpy数组的测试图片数据。
- `endpoint`：`Endpoint`资源在其中部署`Model`资源的 Vertex 完全限定标识符。
- `parameters_dict`：用于提供的额外参数。

这个函数调用预测客户端服务的`predict`方法，带有以下参数：

- `endpoint`：`Endpoint`资源在其中部署`Model`资源的 Vertex 完全限定标识符。
- `instances`：要预测的实例（编码后的图片）列表。
- `parameters`：用于提供的额外参数。

为了将图片数据传递给预测服务，在之前的步骤中，你将字节编码为base64 -- 这样当通过网络传输二进制数据时，内容就不会被修改。你需要告诉提供二进制的位置，你的模型被部署到哪里，内容已经被base64编码，这样它就会在提供二进制的另一端对其进行解码。

预测请求中的每个实例都是以下形式的字典条目：

                        {serving_input: {'b64': content}}

- `input_name`：基础模型的输入层的名称。
- `'b64'`：指示内容已经被base64编码的键。
- `content`：压缩后的JPG图片字节，作为base64编码的字符串。

由于`predict()`服务可以接受多张图片（实例），你将把单张图片作为一个图片列表发送。最后一步，你把实例列表打包成 Google 的 protobuf 格式 -- 这是我们传递给`predict()`服务的内容。

`response`对象返回一个列表，其中列表中的每个元素对应请求中的相应图片。你将看到每个预测的输出中：

- `predictions`：每个类别的预测置信度，介于0和1之间。

In [ ]:
def predict_image(image, endpoint, parameters_dict):
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{serving_input: {"b64": image}}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters_dict
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", prediction)


predict_image(b64str, endpoint_id, None)

## 取消部署 `Model` 资源

现在从服务 `Endpoint` 资源中取消部署您的 `Model` 资源。使用这个辅助函数 `undeploy_model`，该函数接受以下参数：

- `deployed_model_id`：当 `Model` 资源被部署到服务端时由端点服务返回的模型部署标识符。
- `endpoint`：`Model` 被部署到的 Vertex 完全限定标识符。

该函数调用端点客户端服务的方法 `undeploy_model`，带有以下参数：

- `deployed_model_id`：当 `Model` 资源被部署时端点服务返回的模型部署标识符。
- `endpoint`：`Model` 资源部署到的 Vertex 完全限定标识符。
- `traffic_split`：如何在服务 `Endpoint` 资源上的其余部署模型之间分配流量。

由于这是在服务 `Endpoint` 资源上唯一部署的模型，您可以通过将 `traffic_split` 设置为空来简单地将其留空为 {}。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

# 清理工作

要清理此项目中使用的所有GCP资源，可以删除您用于教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的个别资源：

- 数据集
- 流水线
- 模型
- 终端节点
- 批处理作业
- 自定义作业
- 超参数调优作业
- Cloud Storage存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME